In [1]:
import dgl
import torch
from utils_gnn import cnn2graph_activation
# from model_lib import mnist_cnn_model as father_model
from utils_basic import load_spec_model
from utils_gnn import padding, unpadding
from dgl.data import DGLDataset
import os
from tqdm import tqdm
from dgl import save_graphs, load_graphs
import torch
import json
from torch import nn
import numpy as np
# from model_lib.mnist_cnn_model import Model
from random import randint
from utils_basic import load_spec_model
from sklearn.metrics import roc_auc_score
from torchinfo import summary
import torch.nn.functional as F
from dgl.nn.pytorch.conv import GINConv
from dgl.nn.pytorch.glob import SumPooling, AvgPooling, MaxPooling, SortPooling
from utils_gnn import MLP


# x = '/home/dorian/repos/Meta-Nerual-Trojan-Detection/shadow_model_ckpt/mnist/models5/shadow_jumbo_9.model'
x = './shadow_model_ckpt/mnist/models5/shadow_jumbo_9.model'
# load model 
# Model = load_spec_model(father_model, '5')
from model_lib.mnist_cnn_model import Model6 as Model
model = Model(gpu=True)
params = torch.load(x)
model.load_state_dict(params)
del params

# load model detail 
model_detail = {}
model_detail_path = "./intermediate_data/model_detail.json"
import json
with open(model_detail_path, 'r') as f:
    model_detail = json.load(f)
# print(model_detail)
g = cnn2graph_activation(model, model_detail['mnist']['5'])
dgl.save_graphs('./intermediate_data/grapj_test.bin', g)
del model_detail

import torch
import torchvision
import torchvision.transforms as transforms
# from utils_gnn import SGNACT
GPU = True
if GPU:
        torch.cuda.manual_seed_all(0)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
transform = transforms.Compose([
            transforms.ToTensor(),
        ])
BATCH_SIZE = 1
# MNIST image dataset 
trainset = torchvision.datasets.MNIST(root='./raw_data/', train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE)

# get a image
image = None
label = None
for i, (x_in, y_in) in enumerate(dataloader):
    image = x_in
    model(image)
    label = y_in
    break
del trainset, dataloader

In [2]:
# define the init process
def init_conv(data, data_size, weight, bias, kernel_size, stride, padding):
    row, col = weight.size()
    # print(weight.size())
    # get actual conv kernel weight and bias
    w = unpadding(weight, kernel_size[0], kernel_size[1])
    w = w.unsqueeze(0).unsqueeze(0)
    b = unpadding(bias, 1, 1)[0]
    # get conv operator 
    # print("kernel_size, stride, padding:")
    # print(kernel_size, stride, padding)
    operator = torch.nn.Conv2d(1, 1, kernel_size=kernel_size, 
                    stride=stride, padding=padding)
    # set conv operator weight and bias
    operator.weight.data = w
    operator.bias.data = b
    # conduct conv operation
    # print("conv input size:", data.size())
    x = operator(data.to("cuda"))

    return x


def initiate_node_feature(graph, image):
    ft = None 
    mask = graph.ndata['tag'] == 0
    out_channels = int(sum(mask))
    ft = torch.zeros((len(graph.nodes()), 1, 1, 28, 28))
    convd_size = torch.zeros((len(graph.nodes()), 2))
    for i in range(out_channels):
        kernel_size, stride, padding = graph.ndata['kernel_params'][i]
        # do conv
        res_ft = init_conv(image, None, graph.ndata['kernel_weight'][i], graph.ndata['bias'][i],
                kernel_size, stride, padding)
        # do relu
        relu_opt = torch.nn.functional.relu
        res_ft = relu_opt(res_ft)

        # do max pooling
        pooling = graph.ndata['pooling_params'][i]
        if pooling.all() != 0:
            # do max_pooling
            kernel_size, stride, pad, dilation, ceil_mode = pooling
            max_pooling_operator = torch.nn.MaxPool2d(kernel_size=kernel_size, stride=stride, 
                padding=pad, dilation=dilation, ceil_mode=ceil_mode)
            
            res_ft = max_pooling_operator(res_ft)
        _, _, r, c = res_ft.size()
        ft[i] = res_ft
        convd_size[i] = torch.tensor([int(r), int(c)])
    graph.ndata['ft'] = ft.to("cuda")
    graph.ndata['ft_size'] = convd_size.to("cuda")

# init ft feature and get subg
with torch.no_grad():
    initiate_node_feature(g, image)
print(g.ndata['tag'])
# get node id
def nodes_with_feature_smaller_two(nodes):
    return nodes.data['tag'] <= 1
nodes_idx = g.filter_nodes(nodes_with_feature_smaller_two)
# print(nodes_idx)
subg = dgl.node_subgraph(g, nodes_idx, relabel_nodes=True)
print(subg.nodes())
ft32 = None
# got the right subg of conv layers

KeyError: 'tag'

In [ ]:
# check init conv out
g.ndata.keys()

dict_keys(['kernel_weight', 'tag', 'layer', 'kernel_size', 'kernel_params', 'bias', 'bias_size', 'pooling_params', 'ft', 'ft_size'])

In [ ]:
mweight = model.conv1.weight.data
gweight = g.ndata['kernel_weight'][:32]

In [ ]:
print(model.conv1.bias.data)

tensor([ 0.0384, -0.1011,  0.1332,  0.0410,  0.1139,  0.0930,  0.0334,  0.1487,
         0.2505, -0.2181, -0.0363,  0.1178,  0.0028, -0.2484,  0.0311, -0.2038,
         0.0550, -0.1416, -0.0180, -0.2482,  0.1559,  0.1873, -0.0378,  0.0986,
        -0.1522, -0.2055, -0.1212,  0.0316, -0.1006,  0.0530, -0.1025, -0.0923],
       device='cuda:0')


In [ ]:
weight = torch.load("./intermediate_data/process/kernel_weight.pt")
print(weight[0][0])

tensor([[-0.0532, -0.0122,  0.0185,  0.0367,  0.0555],
        [-0.0450, -0.0052, -0.0021,  0.0344, -0.0546],
        [ 0.0027,  0.0923,  0.0490, -0.0459, -0.0032],
        [-0.0314,  0.0848,  0.0389,  0.0015,  0.0685],
        [-0.0478, -0.0734,  0.0240, -0.0006,  0.0424]], device='cuda:0')


In [ ]:
from utils_gnn import padding, unpadding, equals
print(mweight.size())
print(gweight.size())
for i in range(32):
    np.savetxt("./intermediate_data/gweight.csv", gweight[i].cpu().numpy(), delimiter=',')
    # gk = unpadding(gweight[i], 5, 5)
    gk = torch.tensor(np.loadtxt("./intermediate_data/init/weight-0.csv", delimiter=','))
    gk = gk.to("cuda")
    print(gk)
    for j in range(32):
        mk = mweight[j][0]
        print(mk)
        if equals(mk, gk):
            print("equals")
    break

torch.Size([32, 1, 5, 5])
torch.Size([32, 512, 512])
tensor([[-0.0891, -0.0668,  0.0155,  0.1121, -0.1723],
        [-0.1256, -0.1489,  0.0532,  0.1980,  0.2342],
        [-0.2169,  0.0664, -0.0389, -0.1413, -0.0992],
        [ 0.1313, -0.1686,  0.2182,  0.1906, -0.1527],
        [ 0.0703,  0.0818,  0.1617, -0.1790,  0.0577]], device='cuda:0',
       dtype=torch.float64)
tensor([[-0.0891, -0.0668,  0.0155,  0.1121, -0.1723],
        [-0.1256, -0.1489,  0.0532,  0.1980,  0.2342],
        [-0.2169,  0.0664, -0.0389, -0.1413, -0.0992],
        [ 0.1313, -0.1686,  0.2182,  0.1906, -0.1527],
        [ 0.0703,  0.0818,  0.1617, -0.1790,  0.0577]], device='cuda:0')
equals
tensor([[-0.0559,  0.1315, -0.1052,  0.0801,  0.1653],
        [-0.2060, -0.2405, -0.1396,  0.0445,  0.3602],
        [ 0.1334, -0.0625,  0.0653,  0.0392,  0.2272],
        [ 0.1069,  0.0004, -0.2087, -0.1554, -0.1788],
        [-0.0611,  0.0623, -0.1455,  0.1574, -0.1059]], device='cuda:0')
tensor([[ 0.0668,  0.0684,  0.035